In [1]:
import numpy as np
import pandas as pd 
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import keras
import shutil
import tensorflow as tf
from keras import backend as K
from keras.layers import Conv2D, MaxPool2D, Dropout, Activation, Dense, Flatten, BatchNormalization
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.preprocessing import image
from keras.metrics import categorical_crossentropy
from keras.layers.convolutional import *
from keras.applications import imagenet_utils
from keras.optimizers import adam
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [11]:
import models

In [10]:
def imshow(x):
    cv2.imshow("image", x)
    cv2.waitKey(5000) # 5 sec delay before image window closes
    cv2.destroyWindow("image")

In [4]:
def resize(img, width, height, interpolation=cv2.INTER_AREA):
    return cv2.resize(img, (width, height), interpolation)

In [5]:
# Loading images from folder
def images_from_folder(folder, label = 1):
    images = []
    labels = []
    for file in os.listdir(folder):
        image = cv2.imread(os.path.join(folder,file))
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = resize(image, 224, 224)
            images.append(image)
            labels.append(label)
    return images, labels

In [7]:
images_from_folder("/Users/kbsriharsha/Documents/Education/Python/Covid19Detection/data/covid")

[array([[[58, 56, 59],
         [58, 56, 59],
         [58, 56, 59],
         ...,
         [58, 56, 59],
         [58, 56, 59],
         [60, 58, 61]],
 
        [[58, 56, 59],
         [58, 56, 59],
         [58, 56, 59],
         ...,
         [58, 56, 59],
         [58, 56, 59],
         [60, 58, 61]],
 
        [[58, 56, 59],
         [58, 56, 59],
         [58, 56, 59],
         ...,
         [58, 56, 59],
         [58, 56, 59],
         [60, 58, 61]],
 
        ...,
 
        [[58, 56, 59],
         [58, 56, 59],
         [58, 56, 59],
         ...,
         [58, 56, 59],
         [58, 56, 59],
         [60, 58, 61]],
 
        [[58, 56, 59],
         [58, 56, 59],
         [58, 56, 59],
         ...,
         [58, 56, 59],
         [58, 56, 59],
         [60, 58, 61]],
 
        [[58, 56, 59],
         [58, 56, 59],
         [58, 56, 59],
         ...,
         [58, 56, 59],
         [58, 56, 59],
         [60, 58, 61]]], dtype=uint8), array([[[68, 69, 71],
         [67, 68, 70

In [11]:
 a = "/".join(os.getcwd().split("/")[:-1] + ["data","covid"])

In [9]:
os.listdir("/".join(os.getcwd().split("/")[:-1] + ["data","covid"]))

['nejmoa2001191_f5-PA.jpeg',
 'nejmc2001573_f1a.jpeg',
 'auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg',
 'ryct.2020200034.fig5-day4.jpeg',
 'lancet-case2a.jpg',
 'nCoV-radiol.2020200269.fig1-day7.jpeg',
 '1-s2.0-S0929664620300449-gr2_lrg-d.jpg',
 'lancet-case2b.jpg',
 'auntminnie-c-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg',
 '1-s2.0-S0929664620300449-gr2_lrg-a.jpg',
 'auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg',
 '1-s2.0-S0929664620300449-gr2_lrg-b.jpg',
 '1-s2.0-S0929664620300449-gr2_lrg-c.jpg',
 'radiol.2020200490.fig3.jpeg',
 'ryct.2020200028.fig1a.jpeg',
 '1-s2.0-S0140673620303706-fx1_lrg.jpg',
 'nejmoa2001191_f3-PA.jpeg',
 'radiopedia-covid-19-pneumonia-2.jpg',
 'ryct.2020200034.fig5-day7.jpeg',
 'nejmoa2001191_f4.jpeg',
 'nejmc2001573_f1b.jpeg',
 'nejmoa2001191_f1-PA.jpeg',
 'ryct.2020200034.fig5-day0.jpeg',
 'auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg',
 'ryct.2020200034.fig2.jpeg'

In [6]:
covid_features, covid_labels = images_from_folder("/".join(os.getcwd().split("/")[:-1] + ["data","covid"]))

In [8]:
len(covid_labels)

100

In [7]:
noncovid_features, noncovid_labels = images_from_folder("/".join(os.getcwd().split("/")[:-1] + ["data","non_covid"]), label = 0)

In [9]:
len(noncovid_labels)

100

In [108]:
noncovid_labels

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
images = covid_features + noncovid_features
labels = covid_labels + noncovid_labels

In [111]:
len(images)

50

In [115]:
data = np.asarray(images)
labels1 = np.asarray(labels)

In [116]:
labels1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [117]:
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    '''
    if v2:
        x = x - 0.5
        x = x * 2.0
    '''
    return x

In [118]:
preprocess_input(data)

array([[[[0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255],
         ...,
         [0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255],
         [0.23529412, 0.22352941, 0.23529412]],

        [[0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255],
         ...,
         [0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255],
         [0.23529412, 0.22745098, 0.23529412]],

        [[0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255],
         ...,
         [0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255],
         [0.23529412, 0.22745098, 0.23529412]],

        ...,

        [[0.22745098, 0.21960784, 0.23137255],
         [0.22745098, 0.21960784, 0.23137255]

In [119]:
labels = keras.utils.to_categorical(labels)

In [120]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [121]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, random_state=42)

In [122]:
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [90]:
labels

array([[[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],


        [[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ...

In [1]:
import models

Using TensorFlow backend.


In [2]:
# Build MOdel
model = models.mobilenet(num_classes = 2)

Instructions for updating:
Colocations handled automatically by placer.


TypeError: Output tensors to a Model must be Keras tensors. Found: <keras.engine.training.Model object at 0x643b95b00>

In [123]:
model = keras.applications.mobilenet.MobileNet(include_top=False, weights= 'imagenet', input_shape= (224,224,3))

In [124]:
# Defining model
def mobilenet():
    model = keras.applications.mobilenet.MobileNet(include_top=False, weights= 'imagenet', input_shape= (224,224,3))
    new_output = keras.layers.GlobalAveragePooling2D()(model.output)
    new_output = keras.layers.Dense(2, activation='softmax')(new_output)
    model = keras.engine.training.Model(model.inputs, new_output)
    return model

In [125]:
model = mobilenet()

In [77]:

len(model.layers)

84

In [126]:
for layer in model.layers[:-23]:
        layer.trainable = False

In [127]:
# Compiling model
model.compile(
    loss='categorical_crossentropy',  # we train 102-way classification
    optimizer=keras.optimizers.adamax(lr=1e-2),  # we can take big lr here because we fixed first layers
    metrics=['accuracy']  # report accuracy during training
)

In [128]:
INIT_LR = 1e-3
EPOCHS = 25
BS = 8

In [83]:
len(trainX[0])

224

In [84]:
trainY

array([[[[  3,   3,   3],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  1,   1,   1],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  1,   1,   1],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        ...,

        [[  0,   0,   0],
         [  0,   0,   0],
         [  1,   1,   1],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  1,   1,   1],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  4,   4,   4],
         ...,
         [  0,   0,   0],
        

In [129]:
model.fit_generator(data_generator.flow(trainX,trainY,8),
                        steps_per_epoch=len(trainX) / 8,
                        epochs=25, verbose=1)

Epoch 1/25
5/5 [==============================] - 14s 3s/step - loss: 0.9156 - acc: 0.7750
Epoch 2/25
5/5 [==============================] - 4s 751ms/step - loss: 1.8204 - acc: 0.8000
Epoch 3/25
5/5 [==============================] - 4s 762ms/step - loss: 0.3950 - acc: 0.9500
Epoch 4/25
5/5 [==============================] - 4s 796ms/step - loss: 0.3801 - acc: 0.9500
Epoch 5/25
5/5 [==============================] - 4s 827ms/step - loss: 0.6347 - acc: 0.9500
Epoch 6/25
5/5 [==============================] - 4s 790ms/step - loss: 0.0635 - acc: 0.9750
Epoch 7/25
5/5 [==============================] - 4s 755ms/step - loss: 0.0571 - acc: 0.9750
Epoch 8/25
5/5 [==============================] - 4s 754ms/step - loss: 0.1005 - acc: 0.9250
Epoch 9/25
5/5 [==============================] - 4s 754ms/step - loss: 0.2614 - acc: 0.9250
Epoch 10/25
5/5 [==============================] - 4s 754ms/step - loss: 0.0358 - acc: 1.0000
Epoch 11/25
5/5 [==============================] - 4s 752ms/step - loss

In [ ]:
model.fit_generator(train_bches, steps_per_epoch=15, validation_data=valid_bches, validation_steps= 3, epochs = 40,verbose=1)